In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
from top2vec import Top2Vec
import sys
sys.path.insert(1, '../../rtsvg')
from rtsvg import *
rt = RACETrack()

In [ ]:
import os
_base_dir_ = '../../../data/2014_vast/MC1/News Articles'
_lu_ = {'directory':[], 'file':[], 'source':[], 'title':[], 'published':[], 'article':[], 'author':[], 'location':[]}
for _news_source_ in os.listdir(_base_dir_):
    for _article_ in os.listdir(os.path.join(_base_dir_, _news_source_)):
        _txt_raw_     = open(os.path.join(_base_dir_, _news_source_, _article_), 'rb').read()        
        _txt_better_  = str(_txt_raw_, encoding='latin').split('\n')
        _txt_         = []
        for _line_ in _txt_better_:
            if _line_ == '' or _line_ == '\n' or _line_ == '\r': continue
            _txt_.append(_line_.strip())        
        _just_article_txt_, _title_, _source_, _published_, _author_, _location_ = [], None, None, None, None, None
        row_i = 0
        while row_i < len(_txt_):
            _row_ = _txt_[row_i]
            if   _row_.startswith('TITLE:'):     _title_     = _row_.replace('TITLE: ', '')
            elif _row_.startswith('SOURCE:'):    _source_    = _row_.replace('SOURCE: ', '')
            elif _row_.startswith('AUTHOR:'):    _author_    = _row_.replace('AUTHOR: ', '')
            elif _row_.startswith('LOCATION:'):  _location_  = _row_.replace('LOCATION: ', '')
            elif _row_.startswith('PUBLISHED:'): 
                _published_ = _row_.replace('PUBLISHED: ', '')
                if _published_ == 'Petrus Gerhard' or _published_ == 'By Haneson Ngohebo':
                    _published_ = _txt_[row_i+1]
                    row_i += 1
                if _published_ == 'PROTESTS SWELL IN ABILA IN RESPONSE TO CALLS TO ACTION':
                    _published_ = _author_
                    _author_    = _title_
                    _title_     = 'PROTESTS SWELL IN ABILA IN RESPONSE TO CALLS TO ACTION'
            else:                                
                _just_article_txt_.append(_row_)
            row_i += 1
            
        _lu_['directory'].append(_news_source_), _lu_['file'].append(_article_), _lu_['author'].append(_author_), _lu_['location'].append(_location_)
        _lu_['title'].append(_title_), _lu_['source'].append(_source_), _lu_['published'].append(_published_), _lu_['article'].append('\n'.join(_just_article_txt_))

df = pd.DataFrame(_lu_)
df = rt.columnsAreTimestamps(df, 'published')

In [ ]:
ls = df['article'].to_list()
topic_model = Top2Vec(ls)
df['topic_number'] = topic_model.get_documents_topics(list(range(len(df))))[0]
print(topic_model.get_num_topics())
rt.smallMultiples(df, category_by='topic_number', sm_type='wordCloud', sm_params={'text_fields':'article'})

In [ ]:
rt.tile([rt.temporalBarChart(df, w=1024, h=300), rt.histogram(df, bin_by='topic_number', color_by='source', w=300, h=300)])

In [ ]:
rt.xy(df, x_field='published', y_field=['topic_number','source'], color_by='source', w=1024, h=384)